# Figure 5
## Comparison of figure generation with and without Declarative interface
Adapted from https://github.com/Unidata/python-training/blob/master/pages/gallery/Declarative_300hPa.ipynb.
As an addendum, we generate the identical figure without Declarative. These code snippets are used to generate Figure 4.

### With Declarative
Here we generate the example figure using MetPy's Declarative plotting interface.

In [ ]:
import xarray as xr

import metpy.calc as mpcalc

# get_test_data is used for internal MetPy testing and not supported publicly
from metpy.cbook import get_test_data
from metpy.units import units

Read GFS output valid 1200 UTC 31 October 2010, provided as part of MetPy's internal testing data.

In [ ]:
data = xr.open_dataset(get_test_data("GFS_test.nc", False)).metpy.parse_cf().squeeze()

In [ ]:
dt_string = data["time"].dt.strftime("%Y-%m-%d %T").data

In [ ]:
ds = data.metpy.sel(lat=slice(70, 10), lon=slice(360 - 150, 360 - 55))

In [ ]:
ds["wind_speed"] = mpcalc.wind_speed(
    ds["u-component_of_wind_isobaric"], ds["v-component_of_wind_isobaric"]
)

In [ ]:
from metpy.plots import BarbPlot, ContourPlot, FilledContourPlot, MapPanel, PanelContainer

In [ ]:
contour = ContourPlot()
contour.data = ds
contour.field = "Geopotential_height_isobaric"
contour.level = 300 * units.hPa
contour.contours = list(range(0, 10000, 120))
contour.clabels = True

In [ ]:
cfill = FilledContourPlot()
cfill.data = ds
cfill.field = "wind_speed"
cfill.level = 300 * units.hPa
cfill.contours = list(range(10, 201, 20))
cfill.colormap = "BuPu"
cfill.colorbar = "horizontal"
cfill.plot_units = "knot"

In [ ]:
barbs = BarbPlot()
barbs.data = ds
barbs.field = ["u-component_of_wind_isobaric", "v-component_of_wind_isobaric"]
barbs.level = 300 * units.hPa
barbs.skip = (3, 3)
barbs.plot_units = "knot"

In [ ]:
panel = MapPanel()
panel.area = [-125, -74, 20, 55]
panel.projection = "lcc"
panel.layers = ["states", "coastline", "borders"]
panel.title = f"{cfill.level:~P} Heights and Wind Speed at {dt_string}"
panel.plots = [cfill, contour, barbs]

In [ ]:
pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

In [ ]:
pc.save("images/fig5_declarative.png", dpi=600, bbox_inches="tight")

### Without Declarative
Here, we re-create an identical figure to above, but through "imperative" Matplotlib syntax.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

In [ ]:
level = 300 * units.hPa

In [ ]:
ds_subset = ds[
    [
        "Geopotential_height_isobaric",
        "wind_speed",
        "u-component_of_wind_isobaric",
        "v-component_of_wind_isobaric",
    ]
].metpy.sel(isobaric3=level)

In [ ]:
plot_crs = ccrs.LambertConformal(
    central_latitude=40, central_longitude=-100, standard_parallels=[30, 60]
)

In [ ]:
data_crs = ds_subset["metpy_crs"].metpy.cartopy_crs

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(projection=plot_crs)

In [ ]:
c = ax.contour(
    ds_subset["lon"],
    ds_subset["lat"],
    ds_subset["Geopotential_height_isobaric"],
    levels=list(range(0, 10000, 120)),
    transform=data_crs,
    colors="k",
)

In [ ]:
c.clabel(inline=1, fmt="%.0f", inline_spacing=8, use_clabeltext=True)

In [ ]:
cf = ax.contourf(
    ds_subset["lon"],
    ds_subset["lat"],
    ds_subset["wind_speed"].metpy.convert_units("knots"),
    levels=list(range(10, 201, 20)),
    cmap="BuPu",
    transform=data_crs,
)

In [ ]:
fig.colorbar(cf, orientation="horizontal", pad=0, aspect=50)

In [ ]:
x_slice = slice(None, None, 3)
y_slice = slice(None, None, 3)

In [ ]:
ds_subset = ds_subset.sel(lon=x_slice, lat=y_slice)

In [ ]:
ax.barbs(
    ds_subset["lon"],
    ds_subset["lat"],
    ds_subset["u-component_of_wind_isobaric"].metpy.convert_units("knots").data,
    ds_subset["v-component_of_wind_isobaric"].metpy.convert_units("knots").data,
    pivot="middle",
    transform=data_crs,
)

In [ ]:
ax.set_extent((-125, -74, 20, 55), ccrs.PlateCarree())

In [ ]:
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)

In [ ]:
ax.set_title(f"{level:~P} Heights and Wind Speed at {dt_string}")